In [1]:
import numpy as np

from pytensor import tensor as pt
import pytensor

from lifelines.datasets import load_kidney_transplant

from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.util import Surv

In [2]:

def get_breslow_neg_log_likelihood_loss_jacobian_hessian_function_pytensor() -> pytensor.compile.function.types.Function:
    def reverse_cumsum(a):
        return pt.flip(pt.cumsum(pt.flip(a)))
    
    weights  = pt.vector('weights',dtype='float64')
    data = pt.matrix('data',dtype='float64')
    event = pt.vector('event',dtype='float64')
    n_unique_times = pt.scalar('n_unique_times',dtype='int64') 
    time_return_inverse = pt.vector('time_return_inverse',dtype='int64')

    o = pt.dot(data, weights)
    risk_set = reverse_cumsum(pt.bincount(time_return_inverse,weights= pt.exp(o),minlength=n_unique_times))[time_return_inverse]
    loss = - pt.sum(event * (o - pt.log(risk_set)))

    jacobian = pytensor.gradient.jacobian(loss,weights)
    hessian = pytensor.gradient.hessian(loss,weights)
    neg_log_likelihood_loss_jacobian_hessian = pytensor.function(inputs=[weights,data,event,n_unique_times,time_return_inverse],outputs= [loss,jacobian,hessian])

    return neg_log_likelihood_loss_jacobian_hessian


In [3]:
neg_log_likelihood_loss_jacobian_hessian = get_breslow_neg_log_likelihood_loss_jacobian_hessian_function_pytensor()

In [4]:
data = load_kidney_transplant()


In [5]:
X = data[['age','black_male','white_male','black_female']].to_numpy()

def normalize(X):
    return np.subtract(X , X.mean(axis=0))/X.std(axis=0)

X = normalize(X)
time = data['time'].to_numpy()
event = data['death'].to_numpy()
unique_times, time_return_inverse =  np.unique(time,return_inverse=True)
n_unique_times = len(unique_times)

In [6]:
def train_weights_for_cox_ph_breslow(X,event,n_unique_times,time_return_inverse, max_itterations = 100, neg_log_likelihood_loss_jacobian_hessian=neg_log_likelihood_loss_jacobian_hessian):
    #https://myweb.uiowa.edu/pbreheny/7210/f15/notes/10-27.pdf 
    #according to Dr. Breheny's notes, one should start halfsteping Newton-Raphson for cox when one starts having touble traning, before terminating the training loop
    #"Supposedly" R's survival package does this

    
    weights = np.zeros(X.shape[1])

    last_loss = np.array(np.inf)
    
    half_step = False
    
    for i in range(max_itterations):
        loss, jacobian, hessian = neg_log_likelihood_loss_jacobian_hessian(weights,X,event,n_unique_times,time_return_inverse)
        if (loss < last_loss) &  (not half_step):
            last_loss = loss
            weights = weights -  np.dot(np.linalg.inv(hessian),jacobian)
        elif (loss < last_loss) & half_step:
            last_loss = loss
            weights = weights - (0.5 * np.dot(np.linalg.inv(hessian),jacobian))
        else:
            if half_step:
                break
            else:
                half_step = True

    return weights



In [7]:
 train_weights_for_cox_ph_breslow(X,event,n_unique_times,time_return_inverse)

array([ 0.68441213, -0.01099881,  0.04867061,  0.1145396 ])

In [8]:
y_sur = Surv().from_arrays(event,time)
CoxPHSurvivalAnalysis(ties='breslow').fit(X ,y_sur).coef_

array([ 0.68441213, -0.01099881,  0.04867061,  0.1145396 ])

In [9]:
#weights are completely identical

In [10]:
%timeit  train_weights_for_cox_ph_breslow(X,event,n_unique_times,time_return_inverse)

4.23 ms ± 414 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
%timeit CoxPHSurvivalAnalysis(ties='breslow').fit(X ,y_sur)

67.7 ms ± 2.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
#we are 30X- ish faster =), this due to the fact sksurv/lifelines/statsmodels all write huge parts of thier traning loop pure python